In [2]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer



/Users/enjalot/code/latent-scope/testenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/enjalot/code/latent-scope/testenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/enjalot/code/latent-scope/testenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [10]:


model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
print("max length", model.max_seq_length)
sentences = ['clustering: What is TSNE?', 'clustering: Who is Laurens van der Maaten?']
embeddings = model.encode(sentences, convert_to_tensor=True)
# embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
# embeddings = embeddings[:, :matryoshka_dim]
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings)

<All keys matched successfully>


max length 8192
tensor([[ 0.0180,  0.0758, -0.2148,  ..., -0.0548, -0.0409, -0.0224],
        [ 0.0171,  0.0389, -0.2187,  ..., -0.0272, -0.0431,  0.0048]],
       device='mps:0')


In [3]:
model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-small-en", # switch to en/zh for English or Chinese
    # "nomic-ai/nomic-embed-text-v1",
    trust_remote_code=True
)

print(model.max_seq_length)
sentences = ['clustering: What is TSNE?', 'clustering: Who is Laurens van der Maaten?']
embeddings = model.encode(sentences, convert_to_tensor=True)

embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings)

8192
tensor([[-0.0595, -0.0486,  0.0134,  ...,  0.0568,  0.0487,  0.0124],
        [-0.0419, -0.0770, -0.0093,  ...,  0.0772,  0.0247,  0.0045]],
       device='mps:0')


/Users/enjalot/code/latent-scope/testenv/lib/python3.12/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:334.)
  nonzero_finite_vals = torch.masked_select(


In [4]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: JinaBertModel 
  (1): Pooling({'word_embedding_dimension': 512, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [5]:
model.tokenizer

BertTokenizerFast(name_or_path='jinaai/jina-embeddings-v2-small-en', vocab_size=30528, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [13]:
import requests
entries = requests.get(
    "https://huggingface.co/api/models?filter=sentence-transformers&sort=downloads&limit=100&full=true&config=true",
    headers={}
).json()
print(entries)

[{'_id': '621ffdc136468d709f180292', 'id': 'sentence-transformers/all-MiniLM-L12-v2', 'author': 'sentence-transformers', 'gated': False, 'lastModified': '2024-03-26T14:05:34.000Z', 'likes': 150, 'private': False, 'sha': 'a05860a77cef7b37e0048a7864658139bc18a854', 'config': {'architectures': ['BertModel'], 'model_type': 'bert', 'tokenizer_config': {'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}}, 'downloads': 83551602, 'tags': ['sentence-transformers', 'pytorch', 'rust', 'safetensors', 'bert', 'feature-extraction', 'sentence-similarity', 'transformers', 'en', 'dataset:s2orc', 'dataset:flax-sentence-embeddings/stackexchange_xml', 'dataset:ms_marco', 'dataset:gooaq', 'dataset:yahoo_answers_topics', 'dataset:code_search_net', 'dataset:search_qa', 'dataset:eli5', 'dataset:snli', 'dataset:multi_nli', 'dataset:wikihow', 'dataset:natural_questions', 'dataset:trivia_qa', 'dataset:embedding-data/sentence-compression', 'dataset:embe

In [8]:
import h5py
import os

In [9]:

# Define the path to the h5 file
h5_file_path = os.path.expanduser('~/latent-scope-data/datavis-misunderstood/embeddings/embedding-001.h5')
# Open the h5 file and read the embeddings
with h5py.File(h5_file_path, 'r') as f:
    embeddings = f['embeddings'][:]

print(embeddings[0])


[-2.80309543e-02 -5.97576983e-02  4.55670021e-02  8.96274447e-02
  5.03601460e-03 -4.82724570e-02  6.88737407e-02  2.36342568e-02
  3.77672096e-03 -3.25405188e-02  4.28509340e-02  3.81805515e-03
 -5.31081147e-02 -7.47150630e-02 -6.21009897e-03 -7.19648376e-02
 -2.39734631e-02 -3.99969555e-02 -5.33126146e-02 -7.22715333e-02
 -2.61736587e-02 -1.90110952e-02  6.08091131e-02 -3.85383554e-02
  4.38702144e-02 -2.70925034e-02 -5.86808100e-02  8.85105599e-03
  8.28609094e-02  6.80618063e-02 -3.84098962e-02  5.28898463e-02
  2.67057051e-03  4.26723287e-02 -4.61654402e-02  2.32309476e-02
  3.31779830e-02  8.90691299e-03 -3.95588614e-02 -1.90383464e-03
 -1.69181954e-02  5.15877530e-02  1.20886587e-01 -8.02913308e-03
 -5.92477806e-02  1.15374243e-02 -8.83747786e-02  6.93125427e-02
  1.94391385e-02  4.84390631e-02 -4.78394292e-02  2.58148462e-02
 -3.87954712e-02  3.49948257e-02  2.43891515e-02  3.43138613e-02
 -4.29521762e-02  5.19764945e-02 -5.48132882e-02  3.52414064e-02
  3.45590860e-02  7.18536

In [10]:
h5_file_path2 = os.path.expanduser('~/latent-scope-data/datavis-misunderstood/embeddings/embedding-002.h5')
# Open the h5 file and read the embeddings
with h5py.File(h5_file_path2, 'r') as f:
    embeddings2 = f['embeddings'][:]

print(embeddings2[0])

[-2.80309469e-02 -5.97577132e-02  4.55669723e-02  8.96274373e-02
  5.03602298e-03 -4.82724532e-02  6.88737631e-02  2.36342568e-02
  3.77674005e-03 -3.25405225e-02  4.28509302e-02  3.81806167e-03
 -5.31080961e-02 -7.47150406e-02 -6.21013530e-03 -7.19648153e-02
 -2.39734650e-02 -3.99969667e-02 -5.33125959e-02 -7.22715482e-02
 -2.61736829e-02 -1.90110635e-02  6.08091392e-02 -3.85383740e-02
  4.38702293e-02 -2.70925257e-02 -5.86808026e-02  8.85105785e-03
  8.28609318e-02  6.80618212e-02 -3.84098664e-02  5.28898463e-02
  2.67057354e-03  4.26722988e-02 -4.61654291e-02  2.32309122e-02
  3.31780091e-02  8.90693069e-03 -3.95588540e-02 -1.90383545e-03
 -1.69181842e-02  5.15877530e-02  1.20886587e-01 -8.02914891e-03
 -5.92477992e-02  1.15373973e-02 -8.83747786e-02  6.93125576e-02
  1.94391441e-02  4.84390967e-02 -4.78394255e-02  2.58148685e-02
 -3.87954600e-02  3.49948108e-02  2.43891515e-02  3.43138427e-02
 -4.29521762e-02  5.19764945e-02 -5.48132770e-02  3.52414213e-02
  3.45590748e-02  7.18537

In [14]:
import numpy as np
from latentscope.models import get_embedding_model
model = get_embedding_model("🤗-nomic-ai___nomic-embed-text-v1.5")
model.load_model()

<All keys matched successfully>


In [15]:
model

In [20]:
dimensions = 768

In [21]:
query = "The importance of good design"

In [109]:
embeddings = np.array(model.embed([query], dimensions=dimensions))
print(embeddings.shape)
embeddings

(1, 64)


array([[-5.58876693e-02,  2.76087999e-01, -3.52094471e-01,
        -5.52555509e-02,  3.52022648e-02,  1.22543119e-01,
         9.53684226e-02,  4.83659618e-02, -1.23020656e-01,
        -6.45322576e-02,  2.78777268e-04, -1.08630978e-01,
         2.34810695e-01,  6.36084974e-02,  1.32510692e-01,
         1.06779739e-01, -5.23752682e-02, -7.33205155e-02,
         9.80967432e-02,  8.23944435e-02, -1.57218501e-01,
        -1.09844655e-01, -1.16264515e-01, -2.38491111e-02,
         1.37694359e-01,  9.50780436e-02, -6.31413385e-02,
         6.04381002e-02, -1.44136593e-01,  1.62359420e-02,
         9.53528285e-02, -9.91425887e-02, -9.77050811e-02,
        -1.52803808e-01, -1.37687093e-02,  8.79531205e-02,
         4.42834161e-02,  2.31125146e-01,  8.55746493e-02,
        -1.80861264e-01,  5.02762683e-02, -3.12233232e-02,
        -5.60627952e-02, -1.13945864e-01,  1.81698978e-01,
        -1.11984931e-01, -5.61883673e-02, -1.37867674e-01,
         1.10347919e-01, -1.57315701e-01,  1.35170639e-0

In [116]:
# confirmed that dimensions=None returns the full embedding which matches whats stored in file
# qemb = np.array(model.embed([query], dimensions=None))
# print(qemb.shape)
# qemb

In [111]:
dimensions = 64

In [117]:
matroyshka = embeddings[:, :dimensions]
matroyshka

array([[-5.58876693e-02,  2.76087999e-01, -3.52094471e-01,
        -5.52555509e-02,  3.52022648e-02,  1.22543119e-01,
         9.53684226e-02,  4.83659618e-02, -1.23020656e-01,
        -6.45322576e-02,  2.78777268e-04, -1.08630978e-01,
         2.34810695e-01,  6.36084974e-02,  1.32510692e-01,
         1.06779739e-01, -5.23752682e-02, -7.33205155e-02,
         9.80967432e-02,  8.23944435e-02, -1.57218501e-01,
        -1.09844655e-01, -1.16264515e-01, -2.38491111e-02,
         1.37694359e-01,  9.50780436e-02, -6.31413385e-02,
         6.04381002e-02, -1.44136593e-01,  1.62359420e-02,
         9.53528285e-02, -9.91425887e-02, -9.77050811e-02,
        -1.52803808e-01, -1.37687093e-02,  8.79531205e-02,
         4.42834161e-02,  2.31125146e-01,  8.55746493e-02,
        -1.80861264e-01,  5.02762683e-02, -3.12233232e-02,
        -5.60627952e-02, -1.13945864e-01,  1.81698978e-01,
        -1.11984931e-01, -5.61883673e-02, -1.37867674e-01,
         1.10347919e-01, -1.57315701e-01,  1.35170639e-0

In [118]:
# Normalize the truncated embeddings
matroyshka = matroyshka / np.linalg.norm(matroyshka, axis=1, keepdims=True)
matroyshka

array([[-5.58876668e-02,  2.76087987e-01, -3.52094455e-01,
        -5.52555484e-02,  3.52022632e-02,  1.22543113e-01,
         9.53684182e-02,  4.83659595e-02, -1.23020651e-01,
        -6.45322547e-02,  2.78777256e-04, -1.08630973e-01,
         2.34810684e-01,  6.36084945e-02,  1.32510686e-01,
         1.06779734e-01, -5.23752658e-02, -7.33205121e-02,
         9.80967387e-02,  8.23944397e-02, -1.57218494e-01,
        -1.09844650e-01, -1.16264509e-01, -2.38491100e-02,
         1.37694353e-01,  9.50780393e-02, -6.31413356e-02,
         6.04380975e-02, -1.44136586e-01,  1.62359413e-02,
         9.53528241e-02, -9.91425841e-02, -9.77050766e-02,
        -1.52803801e-01, -1.37687086e-02,  8.79531164e-02,
         4.42834141e-02,  2.31125136e-01,  8.55746454e-02,
        -1.80861256e-01,  5.02762660e-02, -3.12233218e-02,
        -5.60627927e-02, -1.13945859e-01,  1.81698970e-01,
        -1.11984926e-01, -5.61883647e-02, -1.37867668e-01,
         1.10347914e-01, -1.57315694e-01,  1.35170632e-0

In [82]:
# read in the embeddings from embedding_id
import os
import h5py
embedding_dir = os.path.expanduser("~/latent-scope-data/datavis-misunderstood/embeddings")
embedding_id = "embedding-004"
embedding_path = os.path.join(embedding_dir, f"{embedding_id}.h5")
with h5py.File(embedding_path, 'r') as f:
    dataset = f["embeddings"]
    dembeddings = np.array(dataset)
   

In [83]:
dembeddings.shape

(765, 768)

In [128]:
dmatroyshka = dembeddings[:, :dimensions]
dmatroyshka = dmatroyshka / np.linalg.norm(dmatroyshka, axis=1, keepdims=True)

In [129]:
dmatroyshka.shape

(765, 64)

In [130]:
dmatroyshka[663]

array([-0.05537681,  0.2765364 , -0.35152862, -0.05474492,  0.0356959 ,
        0.12302067,  0.09585046,  0.04885722, -0.12249793, -0.06401982,
        0.00077907, -0.10811045,  0.23526707,  0.06409691,  0.13298632,
        0.10725999, -0.05186532, -0.07280672,  0.09857852,  0.08287948,
       -0.15668889, -0.10932384, -0.11574273, -0.02334458,  0.13816888,
        0.0955606 , -0.06262945,  0.0609268 , -0.14360952,  0.01673306,
        0.09583533, -0.09862395, -0.09718669, -0.15227497, -0.01326591,
        0.08843676,  0.04477515,  0.23158225,  0.08605881, -0.18032745,
        0.05076691, -0.03071724, -0.05555222, -0.11342465,  0.1821652 ,
       -0.11146395, -0.05567789, -0.13734181,  0.11082736, -0.15678618,
        0.13564561, -0.12029664, -0.02852165, -0.10207444,  0.17020841,
        0.03295043,  0.0742992 ,  0.04578885, -0.01956528, -0.06976313,
        0.02728822,  0.31612973, -0.08399071,  0.2022678 ])

In [125]:
# this matches qemb
demb = dembeddings[663]
# demb

In [127]:
dm = demb[:dimensions]
dm = dm / np.linalg.norm(dm, axis=0, keepdims=True)
dm


array([-0.05537681,  0.2765364 , -0.35152862, -0.05474492,  0.0356959 ,
        0.12302067,  0.09585046,  0.04885722, -0.12249793, -0.06401982,
        0.00077907, -0.10811045,  0.23526707,  0.06409691,  0.13298632,
        0.10725999, -0.05186532, -0.07280672,  0.09857852,  0.08287948,
       -0.15668889, -0.10932384, -0.11574273, -0.02334458,  0.13816888,
        0.0955606 , -0.06262945,  0.0609268 , -0.14360952,  0.01673306,
        0.09583533, -0.09862395, -0.09718669, -0.15227497, -0.01326591,
        0.08843676,  0.04477515,  0.23158225,  0.08605881, -0.18032745,
        0.05076691, -0.03071724, -0.05555222, -0.11342465,  0.1821652 ,
       -0.11146395, -0.05567789, -0.13734181,  0.11082736, -0.15678618,
        0.13564561, -0.12029664, -0.02852165, -0.10207444,  0.17020841,
        0.03295043,  0.0742992 ,  0.04578885, -0.01956528, -0.06976313,
        0.02728822,  0.31612973, -0.08399071,  0.2022678 ])